simple_XRD.jl
by Hezy Amiel
January 2023
Julia 1.8.5

In [89]:
using Plots
using SpecialFunctions

In [90]:
function Gaussian(x, fwhm)
    σ = fwhm/(2√(2log(2)))
    return @. 1/√(2π)/σ * exp(-x^2/2σ^2)
end

Gaussian (generic function with 1 method)

In [91]:
function Lorentzian(x, fwhm)
    γ = fwhm / 2
    return @. (γ/pi) / (x^2 + γ^2)
end

Lorentzian (generic function with 1 method)

In [98]:
function Pseudo_Voigt(x, fwhm, n)
	return n * Lorentzian(x, fwhm) + (1 - n) * Gaussian(x, fwhm)
end

Pseudo_Voigt (generic function with 1 method)

In [93]:
function Voigt(x, fwhm_L, fwhm_G)
    γ = fwhm_L/2
    σ = fwhm_G/(2√(2log(2)))
    z = @. -im * (x + im * γ) / (√2 * σ)
    return @. real(erfcx(z)) / (√(2pi) * σ)
end

Voigt (generic function with 1 method)

In [94]:
function find_d(indices, a)
    return a./.√(sum(indices.^2, dims=2))
end

find_d (generic function with 1 method)

In [96]:
N = 100
θ = LinRange(0, 180, N)

wavelength = 0.15418  # CuKα radiation in nm
#wavelength = 0.18125  # 
U, V, W = 0.2, 0.2, 0.2

(0.2, 0.2, 0.2)

In [99]:
y1 = Lorentzian(θ, 1)
y2 = Gaussian(θ, 1)
y3 = Pseudo_Voigt(θ, 1, 0.5)
y4 = Voigt(θ, 0.45, 0.72)

100-element Vector{Float64}:
 0.7899177049600079
 0.023375411437285096
 0.0055131659151482215
 0.0024260532000792063
 0.0013599864296867969
 0.0008690197002484709
 0.0006029705490221322
 0.000442770981633195
 0.00033888349684810796
 0.0002676986240011588
 ⋮
 2.6162478594551147e-6
 2.5596815443442577e-6
 2.5049301404434954e-6
 2.451916828757901e-6
 2.400568812065984e-6
 2.3508170648729103e-6
 2.3025961013148577e-6
 2.25584375955663e-6
 2.2105010013565085e-6

#Simple Cubic
In simple cubic lattince, all Miller indices are allowed

In [100]:
indices = [[h,k,l] for h=-2:2 for k=-2:2 for l=-2:2 if [h,k,l]!=[0,0,0]]

124-element Vector{Vector{Int64}}:
 [-2, -2, -2]
 [-2, -2, -1]
 [-2, -2, 0]
 [-2, -2, 1]
 [-2, -2, 2]
 [-2, -1, -2]
 [-2, -1, -1]
 [-2, -1, 0]
 [-2, -1, 1]
 [-2, -1, 2]
 ⋮
 [2, 1, -1]
 [2, 1, 0]
 [2, 1, 1]
 [2, 1, 2]
 [2, 2, -2]
 [2, 2, -1]
 [2, 2, 0]
 [2, 2, 1]
 [2, 2, 2]